# 추정2

### (1) 유로문제

벨기에 1유로 동전으로 실험을 했는데, 축을 중심으로 250번 회전을 시켰을 때 앞면은 140회 <br />
뒷면은 110회 나왔다. 이 데이터로 동전이 한 쪽으로 기울었다는 것을 증명할 수 있을까?

이것을 증명하기 위해서는 2단계를 거쳐야 한다.
- 동전의 앞면이 나올 확률을 추정하는 것
- 데이터가 동전이 기울었다는 가설을 지지하는지 평가하는 것

#### 전체 코드

In [2]:
import thinkbayes2
import thinkplot

class Euro(thinkbayes2.Suite):
    """Represents hypotheses about the probability of heads."""

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: integer value of x, the probability of heads (0-100)
        data: string 'H' or 'T'
        """
        x = hypo / 100.0 
        if data == 'H':
            return x 
        else:
            return 1-x


class Euro2(thinkbayes2.Suite):
    """Represents hypotheses about the probability of heads."""

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: integer value of x, the probability of heads (0-100)
        data: tuple of (number of heads, number of tails)
        """
        x = hypo / 100.0
        heads, tails = data
        like = x**heads * (1-x)**tails
        return like


def UniformPrior():
    """Makes a Suite with a uniform prior."""
    suite = Euro(range(0, 101))
    return suite


def TrianglePrior():
    """Makes a Suite with a triangular prior."""
    suite = Euro()
    for x in range(0, 51):
        suite.Set(x, x)
    for x in range(51, 101):
        suite.Set(x, 100-x) 
    suite.Normalize()
    return suite


def RunUpdate(suite, heads=140, tails=110):
    """Updates the Suite with the given number of heads and tails.

    suite: Suite object
    heads: int
    tails: int
    """
    dataset = 'H' * heads + 'T' * tails

    for data in dataset:
        suite.Update(data)


def Summarize(suite):
    """Prints summary statistics for the suite."""
    print(suite.Prob(50))

    print('MLE', suite.MaximumLikelihood())

    print('Mean', suite.Mean())
    print('Median', suite.Percentile(50))

    print('5th %ile'
          , suite.Percentile(5)) 
    print('95th %ile', suite.Percentile(95))

    print('CI', suite.CredibleInterval(90))


def PlotSuites(suites, root):
    """Plots two suites.

    suite1, suite2: Suite objects
    root: string filename to write
    """
    thinkplot.Clf()
    thinkplot.PrePlot(len(suites))
    thinkplot.Pmfs(suites)

    thinkplot.Save(root=root,
                   xlabel='x',
                   ylabel='Probability',
                   formats=['pdf', 'eps'])


def main():
    # make the priors
    suite1 = UniformPrior()
    suite1.name = 'uniform'

    suite2 = TrianglePrior()
    suite2.name = 'triangle'

    # plot the priors
    PlotSuites([suite1, suite2], 'euro2')

    # update
    RunUpdate(suite1)
    Summarize(suite1)

    RunUpdate(suite2)
    Summarize(suite2)

    # plot the posteriors
    PlotSuites([suite1], 'euro1')
    PlotSuites([suite1, suite2], 'euro3')


if __name__ == '__main__':
    main()


/Users/choi/anaconda/lib/python3.6/site-packages/matplotlib/axes/_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


Writing euro2.pdf
Writing euro2.eps
0.02097652612954468
MLE 56
Mean 55.952380952380956
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)
0.02384753721469363
MLE 56
Mean 55.74349943859506
Median 56
5th %ile 51
95th %ile 61
CI (51, 61)
Writing euro1.pdf
Writing euro1.eps
Writing euro3.pdf
Writing euro3.eps


주어진 동전을 돌렸을 때 앞면이 나오면서 넘어갈 확률을 x라고 하자. <br />
x값은 동전의 무게 분포를 비롯한 물리적 성격에 따라 달라질 것이다.

만약 동전의 무게가 완벽하게 균등하다면 x의 값은 50%에 가깝겠지만 <br />
한쪽으로 기울어진 동전이라면 상대적으로 x의 값은 달라질 것이다. <br />
여기에 베이즈 이론을 적용해서 데이터 관측을 통해 x를 추정할 수 있다.

Hx가 앞면이 나올 확률이 0부터 100까지 중 x%라는 가설일 때 각각을 정의해보자. <br />
Hx의 확률은 모든 x에 대해서 동일하다는 균등 사전 분포에서 시작할 것이다.

우도 함수는 상대적으로 쉽다. 만약 Hx가 참이라면, 앞면이 나올 확률은 x/100이고 뒷면이 나올 확률은 100-x/100이다.

In [ ]:
class Euro(Suite):
    def Likelihood(self, data, hypo):
        x = hypo
        if data == 'H':
            return x/100.0
        else:
            return 100 - x/100.0

다음은 스윗을 만들고 갱신하는 코드다.

In [ ]:
    suite = Euro(xrange(0, 101))
    dataset = 'H'*140 + 'T'*110
    
    for data in dataset:
        suite.Update(data)

결과는 아래와 같다. 유로 문제의 균등 사전 확률에 대한 사후 확률의 분포

<img src="./Images/1.png" width=400 />

### (2) 사후 확률 요약하기

사후 확률 분포를 요약하는 방법에는 여러 가지가 있다. <br />
한 가지 방법으로는 사후 확률 분포에 가장 근접한 값을 찾는 것이다.

In [ ]:
def MaximunLikehood(pmf):
    """가장 높은 확률의 값을 구한다"""
    prob, val = max((prob, val) for val, prob in pmf.Items())
    return val

이 값은 모수의 최대 우도 추정값이 된다.

또한 평균과 중간값을 구해서 사후 확률을 요약할 수도 있다.

In [ ]:
    print('Mean', suite.Mean())
    print('Median', thinkbayes2.Percentile(suite, 50))

마지막으로 신뢰구간을 계산할 수도 있다.

In [ ]:
    print('CI', thinkbayes.CredibleInterval(suite, 90))

이제 원점으로 돌아와서, 동전이 평평한지 아닌지 알아보자. <br />
이때 사후 신뢰구간이 50%를 포함하고 있지 않으므로 동전이 평평하지 않다는 것을 알 수 있다.

하지만 우리가 답을 구하고자 하는 본 질문은 <br />
"이 데이터로 동전이 한 쪽으로 기울었다는 것을 증명할 수 있을까?"였다. <br />
이 질문에 대답하기 위해서는 가설에 대해 증거를 갖추고 있다는 것이 어떤 의미인지 자세히 살펴야한다.

우리가 동전이 평평한지 아닌지를 안다는 것을 확률이 50%인지 아닌지에 대입할 수 있다. <br />

In [ ]:
print(suite.Prob(50))

### (3) 사전 분포 범람

앞에서 균등 사전 분포는 좋은 선택이 아니다. <br />
50%에 가까운 x의 값에 더 높은 확률을 부여하고 극단 값에 낮은 확률을 부여하는 <br />
사전 확률을 부여하는 사전 확률을 고르는 것이 더 자연스러워 보인다. 

아래와 같이 삼각 사전 확률 분포를 만들어서 사용해 보자.

In [ ]:
def TrianglePrior():
    suite = Euro()
    for x in range(0, 51):
        suite.Set(x, x)
    for x in range(51, 101):
        suite.Set(x, 100-x)
    suite.Normalize()

유로 문제에 대한 균등 사전 확률과 삼각 사전 확률

<img src="./Images/2.png" width=400 />

- 파란색: 균등 사전 확률
- 하늘색: 삼각 사전 확률

동일한 데이터셋을 사용하여 삼각 사전 확률을 갱신하여 사후 확률을 나타내어 보면 <br />
사전 확률이 상당히 다른데도 불구하고 사후 확률 분포는 매우 유사하다.

<img src="./Images/3.png" width=400 />

이는 사전 분포 범람에 대한 예시라고 볼 수 있는데 <br />
데이터가 충분하다면 서로 다른 사전 확률을 가지고 시작한다고 해도 동일한 사후 확률로 수렴하는 경향이 있다.

### (4) 최적화

속도를 향상시킬 몇 가지 방법을 알아보자.

첫 번째 방법은 스윗을 정규화하는 횟수를 줄이는 것이다. <br />
원 코드에는 한 번 돌 때마다 한 번씩 Update를 호출한다.

In [ ]:
    dataset = 'H'*heads + 'T'*tails
    for data in dataset:
        suite.Update(data)

In [ ]:
Update의 코드는 다음과 같다.

In [ ]:
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        return self.Normalize()

매 업데이트 시 가설을 반복하면서 Normalize를 호출하고 가설을 다시 반복한다. <br />
정규화 전에 업데이트의 모든 내용을 수행하면 시간을 다소 절약할 수 있다.

In [ ]:
Suite에서는 이 과정을 수행하는 UpdateSet이라는 메서드를 제공한다.

In [ ]:
    def UpdateSet(self, dataset):
        for data in dataset:
            for hypo in self.Values():
                like = self.Likelihood(data, hypo)
                self.Mult(hypo, like)
        return self.Normalize()

이 함수는 다음과 같이 호출된다.

In [ ]:
    dataset = 'H'*heads + 'T'*tails
    suite.UpdateSet(dataset)

이 최적화를 통해 속도가 향상되지만, 런타임은 여전히 데이터의 양에 비례한다. <br />
데이터셋을 한 번에 한 회씩 처리하는 대신, 전체 데이터셋을 처리하도록 Likelihood 함수를 다시 작성해서 속도를 좀 더 항샹시킬 수 있다.

원 버전에서 data는 앞면인지 뒷면인지 인코딩된 문자열이다.

In [ ]:
    def Likelihood(self, data, hypo):
        x = hypo / 100.0
        if data == 'H':
            return x
        else:
            return 1-x

혹은 데이터를 두 숫자의 집합(앞면과 뒷면의 수)으로 인코딩 할 수도 있다. 이 경우 Likelihood는 다음과 같다.

In [ ]:
    def Likelihood(self, data, hypo):
        x = hypo / 100.0
        heads, tails = data
        like = x**heads*(1-x)**tails
        return like

이 경우 Update를 호출하는 법은 다음과 같다.

In [ ]:
    heads, tails = 140, 110
    suite.Update((heads, tails))

함수가 여러 제곱 회 반복되는 대신 이 버전은 동전을 몇 번 던지더라도 동일한 횟수 만큼 실행된다.

### (5) 베타 분포

지금까지는 x를 이산적인 값의 집합으로 나타내기 위해 Pmf를 사용했다. <br />
이번에는 연속형 분포, 정확히는 베타 분포를 사용해 볼 것이다.<br />
베타 분포는 0과 1(두 수 모두 포함) 사이에서 정의되므로 비율과 확률을 나타내는 방법으로 적합하다.

#### cf) 베타분포

베타 분포는 알파와 베타에 따라 [0, 1] 사이에서 정의되는 연속 확률 분포이다.

<img src="./Images/4.png" width=400 />

베타 분포는 켤레 사전 분포이다.<br />
만약 x에 대한 사전 분포가 베타 분포이면, 사후 확률 역시 베타 분포이다.

베타 분포의 모양은 알파와 베타, 혹은 alpha와 beta라고 쓰는 두 변수에 따라 달라진다.<br />
만약 사전 분포가 alpha와 beta 변수에 대한 베타 분포고 앞면 h와 뒷면 t의 데이터를 가지고 있다면 <br />
사후 확률은 alpha+h 와 beta+t의 변수에 대한 베타 분포가 될 것이다. <br />
즉 두개를 추가해서 갱신을 한다.

이는 베타 분포가 사전 분포로 적합한 경우에만 동작한다. <br />
다행히도 많은 경우 베타 분포는 좋은 추정값으로 사용되고, 균등 사전 분포의 경우에는 <br />
완벽하게 맞아 떨어진다. (alpha=1, beta=1인 베타 분포는 0부터 1까지 균등하다.)

아래는 베타 분포를 나타내는 클래스이다.

In [ ]:
class Beta(object):
    def __init__(self, alpha=1, beta=1):
        self.alpha = alpha
        self.beta = beta

기본 __init__ 에서는 균등 분포를 만든다. Update에서는 베이지안 갱신을 수행한다.

In [ ]:
    def Update(self, data):
        heads, tails = data
        self.alpha += heads
        self.beta += tails

data는 앞면과 뒷면을 나타내는 정수 쌍이다. <br />
그러므로 이제 유로 문제를 풀 수 있는 다른 방안이 생긴 것이다.

In [ ]:
    beta = thinkbayes.Beta()
    beta.Update((140, 110))
    print(beta.Mean())

Beta는 alpha와 beta를 계산하는 간단한 함수인 Mean을 제공한다.

In [ ]:
    def Mean(self):
        return float(self.alpha) / (self.alpha + self.beta)

Beta에는 베타 분포의 확률 밀도 함수(Pdf)를 구하는 EvalPdf라는 함수도 제공한다.

In [ ]:
    def EvalPdf(self, x):
        return x**(self.alpha-1)*(1-x)**(self.beta-1)

### (6) 토의

이 장에서는 두 개의 다른 사전 분포를 가지는 동일한 문제를 풀면서, 데이터셋이 큰 경우 사전 분포 범람 현상이 생긴다는 것을 알게 되었다. <br />
만약 두 사람이 서로 다른 사전 확률로 시작하는 경우 그 사람들이 데이터를 더 접하면서 보통 사후 확률이 수렴한다는 것을 발견하게 되었다. <br />
어떤 점에서 두 분포 간의 차이가 실제로 유의미하지 않을 정도로 작아진다. <br />
많은 문제에서의 사전 확률의 문제는 결국 데이터에 의해 완화된다. <br />
하지만 모든 경우에서 이런 것은 아니다. 우선 모든 베이지안 분석은 의사 결정을 모델링하는 것에 기반한다. <br />
만약 우리가 동일한 모델을 선택하지 않는다면 우리는 데이터를 다르게 해석할 수 있다. <br />
즉 동일한 데이터에 대해서 우리가 서로 다른 우도를 계산할 수 있고, 그러면 우리의 사후 확률은 수렴하지 않을 것이다. <br />
또한, 베이지안 갱신에서 각 사전 확률에 우도를 곱했으므로 D에 상관없이 p(H)가 0이면 p(H|D) 역시 0이다. <br />
유로 문제에서 x가 50%이하라고 생각했다면 나머지 모든 가설의 확률은 0이므로 나머지 데이터는 고려되지 않을 것이다. <br />
이 관측 결과는 크롬웰의 법칙에 기반한다. <br />
이 법칙은 아주 조금이나마 가능성이 있는 나머지 가설에 대해 0을 사전 확률로 부여하는 것을 피하게 해준다.